In [1]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

  Obtaining dependency information for google-api-python-client from https://files.pythonhosted.org/packages/0b/9a/69b094da2ddce819b905e3d63e3bf8713a519a5cec0b109f39a1e5da8926/google_api_python_client-2.105.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for google-auth-httplib2 from https://files.pythonhosted.org/packages/d3/3d/e4991229886c0d522d9552151a43ff7adcc61e026e60ce8bd508387f84cf/google_auth_httplib2-0.1.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for google-auth-oauthlib from https://files.pythonhosted.org/packages/ce/33/a907b4b67245647746dde8d61e1643ef5d210c88e090d491efd89eff9f95/google_auth_oauthlib-1.1.0-py2.py3-none-any.whl.metadata
     ---------------------------------------- 0.0/96.9 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/96.9 kB ? eta -:--:--
     ---------------- --------------------- 41.0/96.9 kB 487.6 kB/s eta 0:00:01
     ---------------------------------------- 96.9/96.9 kB 1.1 MB/s eta 0:00:0


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = '1jPhmRrlACY0j06Bh3MD92fFITFC5HIkAHGTk0dVK2Jw'
SAMPLE_RANGE_NAME = 'Página1!A1:C14'


def main():
    creds = None

    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        service = build('sheets', 'v4', credentials=creds)

        # Ler informacoes do Google Sheets
        sheet = service.spreadsheets()
        result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                    range=SAMPLE_RANGE_NAME).execute()
        valores = result['values']
        print(valores)
        
        valores_adicionar = [
            ["Imposto"],
        ]
        
        for i, linha in enumerate(valores):
            if i > 0:
                vendas = linha[1]
                vendas = vendas.replace("R$ ", "").replace(".", "")
                vendas = float(vendas.replace(",", "."))
                imposto = vendas * 0.1
                imposto = f'R$ {imposto}'.replace(".", ",")
                valores_adicionar.append([imposto])
        
        result = sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                            range="C1", valueInputOption="USER_ENTERED",
                              body={'values': valores_adicionar}).execute()
        # adicionar/editar uma informação
        
        # valores_adicionar = [
        #     ["dezembro", 'R$ 127.300,15'],
        #     ["janeiro", "R$ 100.000,00"],
        # ]
        
        # result = sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,
        #                             range="A13", valueInputOption="USER_ENTERED",
        #                               body={'values': valores_adicionar}).execute()

    except HttpError as err:
        print(err)


if __name__ == '__main__':
     main()

[['Mês', 'Vendas', 'Imposto'], ['janeiro', 'R$ 50.668,00', 'R$ 5.066,80'], ['fevereiro', 'R$ 98.475,00', 'R$ 9.847,50'], ['março', 'R$ 92.030,00', 'R$ 9.203,00'], ['abril', 'R$ 63.546,00', 'R$ 6.354,60'], ['maio', 'R$ 24.496,00', 'R$ 2.449,60'], ['junho', 'R$ 69.997,00', 'R$ 6.999,70'], ['julho', 'R$ 35.242,00', 'R$ 3.524,20'], ['agosto', 'R$ 73.105,00', 'R$ 7.310,50'], ['setembro', 'R$ 57.397,00', 'R$ 5.739,70'], ['outubro', 'R$ 30.293,00', 'R$ 3.029,30'], ['novembro', 'R$ 17.345,00', 'R$ 1.734,50'], ['dezembro', 'R$ 127.300,15', 'R$ 12.730,02'], ['janeiro', 'R$ 100.000,00', 'R$ 10.000,00']]
